In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import json
from datetime import datetime


from bs4 import BeautifulSoup
from key import sbi_id, sbi_pass


In [3]:
options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")


driver = webdriver.Chrome(options=options)

driver.get("https://www.sbisec.co.jp/ETGate")

id_input = driver.find_element(By.NAME, "user_id")
# time.sleep(1)
id_input.send_keys(sbi_id)

pass_input = driver.find_element(By.NAME, "user_password")
# time.sleep(1)
pass_input.send_keys(sbi_pass)

driver.find_element(By.NAME, "ACT_login").click()

In [4]:
link = driver.find_element(By.CSS_SELECTOR, "a img[alt='外国株式・海外ETF']")
driver.execute_script("arguments[0].click();", link)
time.sleep(0.3)

In [5]:
# JavaScriptを使用してクリックする
link = driver.find_element(By.CSS_SELECTOR, "a._mypage_btn__8wnX_")
driver.execute_script("arguments[0].click();", link)
time.sleep(0.3)

In [6]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)
time.sleep(0.5)

In [7]:
input_element = driver.find_element(By.CLASS_NAME, "input-sm")
input_element.send_keys("NVDA")
time.sleep(0.3)

In [8]:
search_button = driver.find_element(By.CSS_SELECTOR, "button[data-ga-button='brandsearch']")

search_button.click()
time.sleep(0.3)

In [9]:
# 'ニュース'タブに対応するボタンを見つける
news_tab = driver.find_element(By.CSS_SELECTOR, "button[data-ga-tab='news']")

# ボタンをクリックする
news_tab.click()
time.sleep(0.3)

In [10]:
iframe = driver.page_source

soup = BeautifulSoup(iframe, "html.parser")
url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"

# Search for iframes or similar tags that might contain the specified URL pattern
iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

# Display the found URLs from iframes
iframe_urls

['https://graph.sbisec.co.jp/sbinews/pc?symbol=NVDA&token=2B18F354FFE8701E96D35A6ECDAD7E72B59CFE5C62EC60F0254E71AF0D9757D20305776A7678265680984B4E64040BC8D4E118CD00CF682735FE0CA2C0C0E9206F86523A40329AA12A6F42DE601DCF00']

In [11]:
print(iframe_urls[0])
driver.execute_script(f"window.open('{iframe_urls[0]}', '_blank');")

https://graph.sbisec.co.jp/sbinews/pc?symbol=NVDA&token=2B18F354FFE8701E96D35A6ECDAD7E72B59CFE5C62EC60F0254E71AF0D9757D20305776A7678265680984B4E64040BC8D4E118CD00CF682735FE0CA2C0C0E9206F86523A40329AA12A6F42DE601DCF00


In [12]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][1]
driver.switch_to.window(new_window)
time.sleep(0.5)

In [13]:
prev_to_next_button = driver.find_element(By.CSS_SELECTOR, "li.paginationjs-page.paginationjs-last.J-paginationjs-page a")
prev_to_next_button.click()


In [14]:
# print(driver.page_source)

<html><head>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
        <meta name="viewport" content="width=device-width, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no">
        <meta name="msapplication-tap-highlight" content="no">
        <title>News PC</title>
        <link rel="stylesheet" href="css/common_pc.css?v=V1.1.10.20230831a" type="text/css">
        <script type="text/javascript" src="lib/jquery-3.6.0.min.js"></script>     
        <script type="text/javascript" src="lib/pagination.min.js"></script>     
        <script type="text/javascript" src="lib/AppWebViewBridge.js"></script>        
        <script type="text/javascript" src="js/constants.js?v=V1.1.10.20230831a"></script>    
        <script type="text/javascript" src="js/utils.js?v=V1.1.10.20230831a"></script>  
        <script type="text/javascript" src="js/news_pc.js?v=V1.1.10.20230831a" symbol="NVDA" url_ticker_redirect="https://global.sbisec.co.jp/invest/us/stock/XXXX?resource

In [20]:


# 今日の日付を取得してファイル名に使用
today_date = datetime.now().strftime("%Y-%m-%d")
filename = f"./scrape_data/{today_date}_news_data.json"

# WebDriverWaitの設定
wait = WebDriverWait(driver, 10)

def collect_news_data():
    news_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "data-row")))
    news_data = []

    for item in reversed(news_items):
        # ニュース項目をクリック
        wait.until(EC.element_to_be_clickable(item)).click()

        # コンテンツが表示されるのを待つ
        content = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".data-row[is-open='true'] .content")))
        # time.sleep(0.02)  # 展開待ち

        headline = item.find_element(By.CLASS_NAME, "headline").text.strip()
        date_time = item.find_element(By.CLASS_NAME, "date").text.strip()
        content_text = content.text.strip() if content.text.strip() else "No content available"

        news_data.append({"headline": headline, "date": date_time, "content": content_text})

    return news_data

# 初回のニュースデータ収集
all_news_data = collect_news_data()
print(all_news_data)

# 「Previous page」の処理
while True:
    try:
        prev_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#pagination-container1 .paginationjs-prev:not(.disabled)")))
        if "disabled" in prev_button.get_attribute("class"):
            print("Reached the first page, no more previous pages.")
            break
        prev_button.click()
        wait.until(EC.staleness_of(news_items[0]))
        new_data = collect_news_data()
        all_news_data.extend(new_data)
        print(new_data)
    except:
        break


# JSONファイルに保存
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(all_news_data, f, ensure_ascii=False, indent=4)

[{'headline': '話題-動き出した米国株-バイオジェン、オーティス、シグナ', 'date': '2024-02-01 04:05\n自動翻訳', 'content': '((ロイターにより自動化された翻訳です。免責条項をご覧ください。https://bit.ly/rtrsauto))\n\n        * \n      Eikonで個別銘柄の動きを検索する文字列：[STXBZ］\n    \n\n        * \n      The Day Ahead ニュースレター https://refini.tv/3LI4BU7 \n    \n\n        * \n      モーニング・ニュース・コールのニュースレター https://refini.tv/3dKUyB8 \n    \n\n  \n\n    ［\u3000１月３１日\u3000］ - アルファベットがAIコストの上昇を予想したことで、メガ株やチップ株の多くが、この日のうちに予定されている米連邦準備制度理事会（FRB）の重要な政策決定を前に、株価を下げた。[.N]\n \n 東部標準時間13時43分現在、ダウ工業株30種平均は0.03％安の38,454.62。S&P500種株価指数は0.78％安の4,886.59、ナスダック総合株価指数は1.25％安の15,315.292。\n \n S&P500種指数の上昇率トップ3：\n ** ストライカー<SYK.N>、8％上昇\n ** パラマウント・グローバル、7.8％上昇\n ** エドワーズ・ライフサイエンシズ<EW.N>、7.5％上昇\n \n S&P 500種の値上がり率トップ3：\n ** マーケットアクセス、15.5%下落\n ** ロックウェルオートメーション<ROK.N>、15.4%下落\n ** テラダイン、8%下落\n \n NYSEの上昇率トップ3：\n ** アシュランド<ASH.N>、15.5％高\n ** 野村、12.4％高\n ** スピリット・エアロシステムズ<SPR.N>、9.7％高\n \n NYSEの値上がり率トップ2：\n ** オラジェニクス、45.5％安\n ** ニューヨーク・コミュニティ・バンコープ<NYCB.N>、37.2％安\n \n ナスダックの値上がり率トップ3：\n ** 